In [3]:
import wbgapi as wb
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import altair as alt

# Data

In [14]:
raw_data_dir = "../data/raw"
africa_gdp = "africa_gdp.csv"

Method 1

Downloading data for all economies, then filtering for Africa

In [7]:
# Get all economies and filter for African countries
all_countries = wb.economy.DataFrame().reset_index()

In [8]:
all_countries

,id,name,aggregate,longitude,latitude,region,adminregion,lendingType,incomeLevel,capitalCity
0,ABW,Aruba,False,-70.0167,12.51670,LCN,,LNX,HIC,Oranjestad
1,AFE,Africa Eastern and Southern,True,NaN,NaN,,,,,
2,AFG,Afghanistan,False,69.1761,34.52280,MEA,MNA,IDX,LIC,Kabul
3,AFW,Africa Western and Central,True,NaN,NaN,,,,,
4,AGO,Angola,False,13.2420,-8.81155,SSF,SSA,IBD,LMC,Luanda
...,...,...,...,...,...,...,...,...,...,...
261,XKX,Kosovo,False,20.9260,42.56500,ECS,ECA,IDX,UMC,Pristina
262,YEM,"Yemen, Rep.",False,44.2075,15.35200,MEA,MNA,IDX,LIC,Sana'a
263,ZAF,South Africa,False,28.1871,-25.74600,SSF,SSA,IBD,UMC,Pretoria
264,ZMB,Zambia,False,28.2937,-15.39820,SSF,SSA,IDX,LMC,Lusaka


In [9]:
african_regions = ['SSF', 'MEA']  # Sub-Saharan Africa and MENA

# Get list of African country codes
africa_countries = all_countries[
    all_countries['region'].isin(['SSA', 'MEA', 'SSF'])
]['id'].tolist()

Method 2 -> Explicit Listing (More Reliable)

In [12]:
africa_codes = [
    'DZA', 'AGO', 'BEN', 'BWA', 'BFA', 'BDI', 'CMR', 'CPV', 'CAF', 
    'TCD', 'COM', 'COG', 'COD', 'CIV', 'DJI', 'EGY', 'GNQ', 'ERI',
    'SWZ', 'ETH', 'GAB', 'GMB', 'GHA', 'GIN', 'GNB', 'KEN', 'LSO',
    'LBR', 'LBY', 'MDG', 'MWI', 'MLI', 'MRT', 'MUS', 'MAR', 'MOZ',
    'NAM', 'NER', 'NGA', 'RWA', 'STP', 'SEN', 'SYC', 'SLE', 'SOM',
    'ZAF', 'SSD', 'SDN', 'TZA', 'TGO', 'TUN', 'UGA', 'ZMB', 'ZWE'
]

# Get GDP data (current US$)
print("Fetching GDP data for African countries...")
gdp_data = wb.data.DataFrame(
    'NY.GDP.MKTP.CD',  # GDP (current US$)
    africa_codes,
    time=range(1957, 2025),  
    labels=True,  # Use country names instead of codes
    skipBlanks=True,
    numericTimeKeys=True
)
print("All done ✅")

Fetching GDP data for African countries...
All done ✅


In [16]:
path = os.path.join(raw_data_dir, africa_gdp)
with open(path, 'w') as f:
    f.write(gdp_data.to_csv(index=False))

In [17]:
def read_gdp_data(country_codes, start_year, end_year,data_dir, name):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    file_path = os.path.join(data_dir, name)
    if not os.path.isfile(file_path):
        print("Fetching GDP data...")
        df = wb.data.DataFrame(
        'NY.GDP.MKTP.CD',  # GDP (current US$)
        africa_codes,
        time=range(1957, 2025),  
        labels=True,  # Use country names instead of codes
        skipBlanks=True,
        numericTimeKeys=True)
        print("All done ✅")
        with open(file_path, 'w') as f:
            f.write(df.to_csv(index=False))
    else:
        print("Reading data from disk...")
        df = pd.read_csv(file_path)
        print("All done ✅")
    
    return df



In [19]:
country_codes = africa_codes
start = 1957
end = 2025

In [20]:
df = read_gdp_data(country_codes = africa_codes, start_year = start, end_year = end ,data_dir = raw_data_dir, name = africa_gdp)

Reading data from disk...
All done ✅


In [21]:
df.head()

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Zimbabwe,1.053528e+09,1.097207e+09,1.118172e+09,1.160104e+09,1.217759e+09,1.312105e+09,1.282404e+09,1.397715e+09,1.480355e+09,...,1.997325e+10,2.055925e+10,5.103566e+10,3.414167e+10,3.335770e+10,3.198033e+10,4.128767e+10,4.075756e+10,3.587178e+10,4.153941e+10
1,Zambia,6.987397e+08,6.823597e+08,6.792797e+08,7.043397e+08,8.226397e+08,1.061200e+09,1.239000e+09,1.340639e+09,1.573739e+09,...,2.125122e+10,2.095841e+10,2.587360e+10,2.631151e+10,2.330867e+10,1.813776e+10,2.209642e+10,2.916378e+10,2.757796e+10,2.530319e+10
2,Uganda,4.231456e+08,4.416673e+08,4.491582e+08,5.163152e+08,5.892477e+08,8.845023e+08,9.253815e+08,9.672407e+08,1.037379e+09,...,3.238718e+10,2.920399e+10,3.074447e+10,3.292703e+10,3.535306e+10,3.760037e+10,4.052979e+10,4.556533e+10,4.876896e+10,5.391191e+10
3,Tunisia,NaN,8.661554e+08,8.800277e+08,1.026738e+09,1.025867e+09,9.910476e+08,1.040952e+09,1.085714e+09,1.214667e+09,...,4.577949e+10,4.436007e+10,4.216353e+10,4.268650e+10,4.190564e+10,4.249178e+10,4.707323e+10,4.492992e+10,4.820533e+10,5.133229e+10
4,Togo,1.710571e+08,1.784971e+08,1.867458e+08,2.023059e+08,2.345722e+08,2.645055e+08,3.052276e+08,3.272158e+08,3.416916e+08,...,5.755458e+09,6.071168e+09,6.387424e+09,7.029216e+09,6.992654e+09,7.486032e+09,8.541668e+09,8.646453e+09,9.816236e+09,1.065118e+10


# Read West Africa Data

In [25]:
data_dir = '../data/raw'
name = 'west_africa.csv'
start = 1970
end = 2025

In [ ]:
def read_west_africa_data(core_indicators, start, end,data_dir, name):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    file_path = os.path.join(data_dir, name)
    if not os.path.isfile(file_path):
        print("Fetching West Africa data...")
        df = wb.data.DataFrame(
            list(core_indicators.keys()),
            west_africa,
            time=range(start, end),
            labels=True)
        df = df.reset_index()
        year_cols = [c for c in df.columns if c.startswith("YR")]
        df_long = df.melt(
            id_vars=["economy", "Country", "series"],
            value_vars=year_cols,
            var_name="year",
            value_name="value"
        )
        df_long["year"] = df_long["year"].str.replace("YR", "").astype(int)

        df_long = df_long.rename(
            columns={
                "economy": "country_code",
                "Country": "country",
                "series": "indicator",
            }
        )
        print("All done ✅")
        with open(file_path, 'w') as f:
            f.write(df_long.to_csv(index=False))
    else:
        print("Reading data from disk...")
        df_long = pd.read_csv(file_path)
        print("All done ✅")
    
    return df_long

In [28]:
west_africa = ['BEN', 'BFA', 'CPV', 'CIV', 'GMB', 'GHA', 'GIN', 'GNB', 
               'LBR', 'MLI', 'MRT', 'NER', 'NGA', 'SEN', 'SLE', 'TGO']

# Core indicators only
core_indicators = {
    'NY.GDP.PCAP.CD': 'GDP per capita',
    'SP.POP.TOTL': 'Population',
    'SP.DYN.LE00.IN': 'Life expectancy',
    'SE.ADT.LITR.ZS': 'Literacy rate',
    'SH.DYN.MORT': 'Child mortality',
    'IT.NET.USER.ZS': 'Internet users %',
    'SL.UEM.TOTL.ZS': 'Unemployment',
    'EG.ELC.ACCS.ZS': 'Electricity access',
    'SI.POV.DDAY': 'Poverty rate',
    'NY.GDP.MKTP.KD.ZG': 'GDP growth'
}

In [36]:
df = read_west_africa_data(core_indicators = core_indicators, start = start, end = end,data_dir = data_dir, name = name)
df.head()

Fetching West Africa data...
All done ✅


,country_code,country,indicator,year,value
0,TGO,Togo,NY.GDP.PCAP.CD,2010,704.986772
1,SLE,Sierra Leone,NY.GDP.PCAP.CD,2010,685.039673
2,SEN,Senegal,NY.GDP.PCAP.CD,2010,1275.883575
3,NGA,Nigeria,NY.GDP.PCAP.CD,2010,2202.256730
4,NER,Niger,NY.GDP.PCAP.CD,2010,474.425692


In [38]:
print(df.dtypes)
print(df.isna().sum().sort_values(ascending=False).head())

country_code     object
country          object
indicator        object
year              int64
value           float64
dtype: object
value           341
country_code      0
country           0
indicator         0
year              0
dtype: int64


In [42]:
print(df.shape)
print(df["indicator"].nunique())
print(df["country"].nunique())
print(df["year"].min()), print(df["year"].max())

(2240, 5)
10
16
2010
2023


(None, None)